<a href="https://colab.research.google.com/github/yopiazani02/Bangkit/blob/master/bankgkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
from datetime import datetime,date
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [3]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/JKM (1).txt',sep="|")
data

,ID_AGE,ID_TK,KODE_KANTOR,TGL_LAHIR,TGL_KEPESERTAAN,JENIS_KELAMIN,KODE_KLAIM,KET_KLAIM,TGL_AGENDA,TGL_PENETAPAN,TGL_BAYAR,TGL_KEMATIAN,STATUS_AGENDA,STATUS_PENETAPAN,KET_STATUS_AGENDA,X_RINCIAN_MNFT,X_BIAYA_DISETUJUI
0,AGE10573,TK1,B00,05/10/1971,01/08/2011,L,B,MENINGGAL DUNIA PADA SAAT KEPESERTAAN AKTIF,15/12/2015,15/12/2015,15/12/2015,NaN,SELESAI,SELESAI,SELESAI,SANTUNAN BERKALA,4800000.0
1,AGE10573,TK1,B00,05/10/1971,01/08/2011,L,B,MENINGGAL DUNIA PADA SAAT KEPESERTAAN AKTIF,15/12/2015,15/12/2015,15/12/2015,NaN,SELESAI,SELESAI,SELESAI,BIAYA PEMAKAMAN,3000000.0
2,AGE10573,TK1,B00,05/10/1971,01/08/2011,L,B,MENINGGAL DUNIA PADA SAAT KEPESERTAAN AKTIF,15/12/2015,15/12/2015,15/12/2015,NaN,SELESAI,SELESAI,SELESAI,SANTUNAN KEMATIAN,16200000.0
3,AGE10573,TK1,B00,05/10/1971,01/08/2011,L,B,MENINGGAL DUNIA PADA SAAT KEPESERTAAN AKTIF,15/12/2015,15/12/2015,15/12/2015,NaN,SELESAI,SELESAI,SELESAI,BEASISWA PENDIDIKAN,0.0
4,AGE12993,TK5,J00,05/06/1955,01/07/2011,L,B,MENINGGAL DUNIA PADA SAAT KEPESERTAAN AKTIF,23/12/2015,23/12/2015,28/12/2015,NaN,SELESAI,SELESAI,SELESAI,SANTUNAN KEMATIAN,16200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14843,AGE11585,TK14785,N02,NaN,NaN,NaN,2,JKM JAKON,18/12/2015,18/12/2015,18/12/2015,NaN,SELESAI,SELESAI,SELESAI,BEASISWA PENDIDIKAN,0.0
14844,AGE12357,TK14785,N00,NaN,NaN,NaN,2,JKM JAKON,21/12/2015,21/12/2015,22/12/2015,NaN,SELESAI,SELESAI,SELESAI,BEASISWA PENDIDIKAN,0.0
14845,AGE12357,TK14785,N00,NaN,NaN,NaN,2,JKM JAKON,21/12/2015,21/12/2015,22/12/2015,NaN,SELESAI,SELESAI,SELESAI,BIAYA PEMAKAMAN,3000000.0
14846,AGE12357,TK14785,N00,NaN,NaN,NaN,2,JKM JAKON,21/12/2015,21/12/2015,22/12/2015,NaN,SELESAI,SELESAI,SELESAI,SANTUNAN BERKALA,4800000.0


In [4]:
data.isnull().sum()

ID_AGE                   0
ID_TK                    0
KODE_KANTOR              0
TGL_LAHIR               96
TGL_KEPESERTAAN         72
JENIS_KELAMIN           76
KODE_KLAIM               0
KET_KLAIM                0
TGL_AGENDA               0
TGL_PENETAPAN           36
TGL_BAYAR              867
TGL_KEMATIAN         14083
STATUS_AGENDA            0
STATUS_PENETAPAN         0
KET_STATUS_AGENDA        0
X_RINCIAN_MNFT           0
X_BIAYA_DISETUJUI       18
dtype: int64

In [0]:
#remove nan value
data = data.drop(columns=['TGL_KEMATIAN','TGL_BAYAR']) #remove column TGL_KEMATIAN
data = data.dropna() #remove row nan value (because dont have so many NaN value in column)
data

In [13]:
#tanggallahir
def calculate_age(born):
    born = datetime.strptime(born, "%d/%m/%Y").date()
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
data['TGL_LAHIR'] = data['TGL_LAHIR'].apply(calculate_age)
data = data.rename(columns={'TGL_LAHIR': 'UMUR'})
data

,ID_AGE,ID_TK,KODE_KANTOR,UMUR,LAMA_KEPESERTAAN,JENIS_KELAMIN,KODE_KLAIM,KET_KLAIM,LAMA_AGENDA,LAMA_PENETAPAN,STATUS_AGENDA,STATUS_PENETAPAN,KET_STATUS_AGENDA,X_RINCIAN_MNFT,X_BIAYA_DISETUJUI
0,AGE10573,TK1,B00,48,8,L,0,3,4,4,3,3,3,2,4800000.0
1,AGE10573,TK1,B00,48,8,L,0,3,4,4,3,3,3,1,3000000.0
2,AGE10573,TK1,B00,48,8,L,0,3,4,4,3,3,3,3,16200000.0
3,AGE10573,TK1,B00,48,8,L,0,3,4,4,3,3,3,0,0.0
4,AGE12993,TK5,J00,64,8,L,0,3,4,4,3,3,3,3,16200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14779,AGE7613,TK14777,E00,36,8,L,0,3,4,4,3,3,3,0,0.0
14780,AGE12329,TK14781,K15,41,8,L,0,3,4,4,3,3,3,0,0.0
14781,AGE12329,TK14781,K15,41,8,L,0,3,4,4,3,3,3,3,16200000.0
14782,AGE12329,TK14781,K15,41,8,L,0,3,4,4,3,3,3,2,4800000.0


In [0]:
#TGL_KEPESERTAAN
data['TGL_KEPESERTAAN'] = data['TGL_KEPESERTAAN'].apply(calculate_age)
data = data.rename(columns={'TGL_KEPESERTAAN': 'LAMA_KEPESERTAAN'})
data

In [0]:
#TGL_AGENDA
data['TGL_AGENDA'] = data['TGL_AGENDA'].apply(calculate_age)
data = data.rename(columns={'TGL_AGENDA': 'LAMA_AGENDA'})
data

In [0]:
#TGL_PENETAPAN
data['TGL_PENETAPAN'] = data['TGL_PENETAPAN'].apply(calculate_age)
data = data.rename(columns={'TGL_PENETAPAN': 'LAMA_PENETAPAN'})
data

In [10]:
# Instatniate LabelEncoder
le = LabelEncoder()
# LabelEncode Class column of df 
data["KET_KLAIM"] = le.fit_transform(data["KET_KLAIM"])
data

,ID_AGE,ID_TK,KODE_KANTOR,UMUR,LAMA_KEPESERTAAN,JENIS_KELAMIN,KODE_KLAIM,KET_KLAIM,LAMA_AGENDA,LAMA_PENETAPAN,STATUS_AGENDA,STATUS_PENETAPAN,KET_STATUS_AGENDA,X_RINCIAN_MNFT,X_BIAYA_DISETUJUI
0,AGE10573,TK1,B00,05/10/1971,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,SANTUNAN BERKALA,4800000.0
1,AGE10573,TK1,B00,05/10/1971,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,BIAYA PEMAKAMAN,3000000.0
2,AGE10573,TK1,B00,05/10/1971,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,SANTUNAN KEMATIAN,16200000.0
3,AGE10573,TK1,B00,05/10/1971,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,BEASISWA PENDIDIKAN,0.0
4,AGE12993,TK5,J00,05/06/1955,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,SANTUNAN KEMATIAN,16200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14779,AGE7613,TK14777,E00,19/03/1984,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,BEASISWA PENDIDIKAN,0.0
14780,AGE12329,TK14781,K15,23/10/1978,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,BEASISWA PENDIDIKAN,0.0
14781,AGE12329,TK14781,K15,23/10/1978,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,SANTUNAN KEMATIAN,16200000.0
14782,AGE12329,TK14781,K15,23/10/1978,8,L,B,3,4,4,SELESAI,SELESAI,SELESAI,SANTUNAN BERKALA,4800000.0


In [11]:
# LabelEncode Class column of df 
data["KODE_KLAIM"] = le.fit_transform(data["KODE_KLAIM"])
data["STATUS_AGENDA"] = le.fit_transform(data["STATUS_AGENDA"])
data["STATUS_PENETAPAN"] = le.fit_transform(data["STATUS_PENETAPAN"])
data["KET_STATUS_AGENDA"] = le.fit_transform(data["KET_STATUS_AGENDA"])
data["X_RINCIAN_MNFT"] = le.fit_transform(data["X_RINCIAN_MNFT"])
data

,ID_AGE,ID_TK,KODE_KANTOR,UMUR,LAMA_KEPESERTAAN,JENIS_KELAMIN,KODE_KLAIM,KET_KLAIM,LAMA_AGENDA,LAMA_PENETAPAN,STATUS_AGENDA,STATUS_PENETAPAN,KET_STATUS_AGENDA,X_RINCIAN_MNFT,X_BIAYA_DISETUJUI
0,AGE10573,TK1,B00,05/10/1971,8,L,0,3,4,4,3,3,3,2,4800000.0
1,AGE10573,TK1,B00,05/10/1971,8,L,0,3,4,4,3,3,3,1,3000000.0
2,AGE10573,TK1,B00,05/10/1971,8,L,0,3,4,4,3,3,3,3,16200000.0
3,AGE10573,TK1,B00,05/10/1971,8,L,0,3,4,4,3,3,3,0,0.0
4,AGE12993,TK5,J00,05/06/1955,8,L,0,3,4,4,3,3,3,3,16200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14779,AGE7613,TK14777,E00,19/03/1984,8,L,0,3,4,4,3,3,3,0,0.0
14780,AGE12329,TK14781,K15,23/10/1978,8,L,0,3,4,4,3,3,3,0,0.0
14781,AGE12329,TK14781,K15,23/10/1978,8,L,0,3,4,4,3,3,3,3,16200000.0
14782,AGE12329,TK14781,K15,23/10/1978,8,L,0,3,4,4,3,3,3,2,4800000.0


In [14]:
data["JENIS_KELAMIN"] = le.fit_transform(data["JENIS_KELAMIN"])
data

,ID_AGE,ID_TK,KODE_KANTOR,UMUR,LAMA_KEPESERTAAN,JENIS_KELAMIN,KODE_KLAIM,KET_KLAIM,LAMA_AGENDA,LAMA_PENETAPAN,STATUS_AGENDA,STATUS_PENETAPAN,KET_STATUS_AGENDA,X_RINCIAN_MNFT,X_BIAYA_DISETUJUI
0,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,2,4800000.0
1,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,1,3000000.0
2,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,3,16200000.0
3,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,0,0.0
4,AGE12993,TK5,J00,64,8,0,0,3,4,4,3,3,3,3,16200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14779,AGE7613,TK14777,E00,36,8,0,0,3,4,4,3,3,3,0,0.0
14780,AGE12329,TK14781,K15,41,8,0,0,3,4,4,3,3,3,0,0.0
14781,AGE12329,TK14781,K15,41,8,0,0,3,4,4,3,3,3,3,16200000.0
14782,AGE12329,TK14781,K15,41,8,0,0,3,4,4,3,3,3,2,4800000.0


In [0]:
data['KODE_KANTOR'].unique()
data['KODE_KANTOR'].value_counts()

K00    334
B04    304
B00    301
N11    275
L00    253
      ... 
L12      4
G04      4
X08      4
B12      4
W20      4
Name: KODE_KANTOR, Length: 141, dtype: int64

In [0]:
data['JENIS_KELAMIN'].unique()
data['JENIS_KELAMIN'].value_counts()

L    12053
P     2719
Name: JENIS_KELAMIN, dtype: int64

In [0]:
data['KODE_KLAIM'].unique()
data['KODE_KLAIM'].value_counts()

B      11874
B2      1363
JKM      883
B1       664
2         64
Name: KODE_KLAIM, dtype: int64

In [0]:
data['KET_KLAIM'].unique()
data['KET_KLAIM'].value_counts()

3    11842
2     1363
0      841
1      660
Name: KET_KLAIM, dtype: int64

In [0]:
data['STATUS_AGENDA'].unique()
data['STATUS_AGENDA'].value_counts()

SELESAI      13981
BATAL          821
PENETAPAN       25
DISETUJUI       21
Name: STATUS_AGENDA, dtype: int64

In [0]:
data['KET_STATUS_AGENDA'].unique()
data['KET_STATUS_AGENDA'].value_counts()

SELESAI      13981
BATAL          821
PENETAPAN       25
DISETUJUI       21
Name: KET_STATUS_AGENDA, dtype: int64

In [0]:
data['X_RINCIAN_MNFT'].unique()
data['X_RINCIAN_MNFT'].value_counts()

BIAYA PEMAKAMAN        3830
SANTUNAN KEMATIAN      3830
SANTUNAN BERKALA       3829
BEASISWA PENDIDIKAN    3359
Name: X_RINCIAN_MNFT, dtype: int64

In [15]:
data

,ID_AGE,ID_TK,KODE_KANTOR,UMUR,LAMA_KEPESERTAAN,JENIS_KELAMIN,KODE_KLAIM,KET_KLAIM,LAMA_AGENDA,LAMA_PENETAPAN,STATUS_AGENDA,STATUS_PENETAPAN,KET_STATUS_AGENDA,X_RINCIAN_MNFT,X_BIAYA_DISETUJUI
0,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,2,4800000.0
1,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,1,3000000.0
2,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,3,16200000.0
3,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,0,0.0
4,AGE12993,TK5,J00,64,8,0,0,3,4,4,3,3,3,3,16200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14779,AGE7613,TK14777,E00,36,8,0,0,3,4,4,3,3,3,0,0.0
14780,AGE12329,TK14781,K15,41,8,0,0,3,4,4,3,3,3,0,0.0
14781,AGE12329,TK14781,K15,41,8,0,0,3,4,4,3,3,3,3,16200000.0
14782,AGE12329,TK14781,K15,41,8,0,0,3,4,4,3,3,3,2,4800000.0


In [0]:
train_dataset = data.sample(frac=0.8,random_state=0)
test_dataset = data.drop(train_dataset.index)

In [18]:
test_dataset

,ID_AGE,ID_TK,KODE_KANTOR,UMUR,LAMA_KEPESERTAAN,JENIS_KELAMIN,KODE_KLAIM,KET_KLAIM,LAMA_AGENDA,LAMA_PENETAPAN,STATUS_AGENDA,STATUS_PENETAPAN,KET_STATUS_AGENDA,X_RINCIAN_MNFT,X_BIAYA_DISETUJUI
0,AGE10573,TK1,B00,48,8,0,0,3,4,4,3,3,3,2,4800000.0
13,AGE2913,TK13,K09,56,8,0,0,3,4,4,3,3,3,1,2000000.0
21,AGE7693,TK21,N11,64,8,0,0,3,4,4,3,3,3,2,4800000.0
25,AGE14029,TK25,C02,29,8,0,0,3,4,4,3,3,3,1,3000000.0
26,AGE14029,TK25,C02,29,8,0,0,3,4,4,3,3,3,3,16200000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14748,AGE9521,TK14749,K15,53,8,0,0,3,4,4,3,3,3,3,14200000.0
14749,AGE9521,TK14749,K15,53,8,0,0,3,4,4,3,3,3,2,4800000.0
14759,AGE13705,TK14757,K12,42,8,0,0,3,4,4,3,3,3,3,16200000.0
14762,AGE5953,TK14761,J04,57,8,0,0,3,4,4,3,3,3,3,14200000.0
